<a href="https://colab.research.google.com/github/Edwin2711/DetectionAlgorithm/blob/main/PruebaAlt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

import cv2
import pathlib
from PIL import Image
from io import open
import torch.functional as F
from torchvision.models import squeezenet1_1

In [ ]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [ ]:
!unzip /content/tanks.zip

Archive:  /content/tanks.zip
   creating: tanks/pred/
  inflating: tanks/pred/c1.jpg       
  inflating: tanks/pred/c5.jpg       
  inflating: tanks/pred/c6.jpg       
  inflating: tanks/pred/s2.jpg       
  inflating: tanks/pred/s5.jpg       
  inflating: tanks/pred/s6.jpg       
   creating: tanks/test/
   creating: tanks/test/composite/
  inflating: tanks/test/composite/c1.jpg  
  inflating: tanks/test/composite/c2.jpg  
  inflating: tanks/test/composite/c3.jpg  
  inflating: tanks/test/composite/c4.jpg  
  inflating: tanks/test/composite/c5.jpg  
  inflating: tanks/test/composite/c6.jpg  
   creating: tanks/test/steel/
  inflating: tanks/test/steel/s1.jpg  
  inflating: tanks/test/steel/s2.jpg  
  inflating: tanks/test/steel/s3.jpg  
  inflating: tanks/test/steel/s4.jpg  
  inflating: tanks/test/steel/s5.jpg  
  inflating: tanks/test/steel/s6.jpg  
   creating: tanks/train/
   creating: tanks/train/composite/
  inflating: tanks/train/composite/c1.jpg  
  inflating: tanks/train/comp

In [ ]:
#Dataloader

#Path for training and testing directory
train_path='tanks/train'
test_path='tanks/test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [ ]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['composite', 'steel']


In [ ]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=2):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (32,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (32,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (32,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (32,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (32,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (32,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (32,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (32,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (32,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (32,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)

        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [ ]:
model=ConvNet(num_classes=2).to(device)

In [ ]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [ ]:
num_epochs=20
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))
print(train_count,test_count)

20 12


In [ ]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count

    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(0.7193) Train Accuracy: 0.45 Test Accuracy: 0.5
Epoch: 1 Train Loss: tensor(11.5041) Train Accuracy: 0.75 Test Accuracy: 0.5833333333333334
Epoch: 2 Train Loss: tensor(0.6264) Train Accuracy: 0.95 Test Accuracy: 0.4166666666666667
Epoch: 3 Train Loss: tensor(2.1582) Train Accuracy: 0.85 Test Accuracy: 0.4166666666666667
Epoch: 4 Train Loss: tensor(0.4938) Train Accuracy: 0.95 Test Accuracy: 0.4166666666666667
Epoch: 5 Train Loss: tensor(1.7881e-08) Train Accuracy: 1.0 Test Accuracy: 0.4166666666666667
Epoch: 6 Train Loss: tensor(0.) Train Accuracy: 1.0 Test Accuracy: 0.4166666666666667
Epoch: 7 Train Loss: tensor(0.) Train Accuracy: 1.0 Test Accuracy: 0.4166666666666667
Epoch: 8 Train Loss: tensor(0.) Train Accuracy: 1.0 Test Accuracy: 0.4166666666666667
Epoch: 9 Train Loss: tensor(0.) Train Accuracy: 1.0 Test Accuracy: 0.4166666666666667
Epoch: 10 Train Loss: tensor(0.) Train Accuracy: 1.0 Test Accuracy: 0.5
Epoch: 11 Train Loss: tensor(0.) Train Accuracy: 

In [ ]:
pred_path='tanks/pred'
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=2)
model.load_state_dict(checkpoint)
model.eval()

#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [ ]:
#prediction function
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred
    

In [ ]:
images_path=glob.glob(pred_path+'/*.jpg')
pred_dict={}

for i in images_path:
    print(i)
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)
pred_dict

tanks/pred/c5.jpg
tanks/pred/s6.jpg
tanks/pred/s2.jpg
tanks/pred/c6.jpg
tanks/pred/s5.jpg
tanks/pred/c1.jpg


{'c5.jpg': 'composite',
 's6.jpg': 'steel',
 's2.jpg': 'steel',
 'c6.jpg': 'composite',
 's5.jpg': 'composite',
 'c1.jpg': 'composite'}